In [ ]:
%pip install matplotlib pandas seaborn scikit-learn dtale seaborn xgboost

- ## 0 **Import de librerias a utilizar**

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("./src"))

import dtale
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import xgboost as xgb


from utils import fraud_licit_ratio, calcular_nulos, split_by_suffix, nan_ratio, visualize_missing_values, missing_audit
from pipelines import split_features , logistic_pipeline_v2, svm_pipeline_v2, xgboost_pipeline_v2

from evaluate import evaluate_multiple
from feature_engineering import build_preprocessor




- ### 0.1 Import de Datasets

In [ ]:
df_train_transaction = pd.read_csv('./dataset/train_transaction.csv')
df_train_identity = pd.read_csv('./dataset/train_identity.csv')
df_test_transaction = pd.read_csv('./dataset/test_transaction.csv')

- ## 1 **Preparacion inicial de los datasets**

> Nuestros datasets esta dividido en dos partes, uno tiene mucha informacion de identidad y otro la data de la transaccion. No todas las transacciones tienen su correspondiente en el Dataset Identity asi que hacemos un merge outer para mantener todos los datos en un dataset grande.

> Utilizamos una herramienta para analisis EDA rapido para visualizar y navegar un dataset que tiene mas de 400 columnas.

In [ ]:
df_full_transaction = pd.merge(
    df_train_transaction,
    df_train_identity,
    on="TransactionID",
    how="outer"
)

print("Shape merged Outer:", df_full_transaction.shape)
dtale.show(df_full_transaction)

In [ ]:
calcular_nulos(df_full_transaction)
fraud_licit_ratio(df_full_transaction)

In [ ]:
nan_ratio(df_train_transaction, 'Transanciones')

nan_ratio(df_train_identity, 'Entidades')

> En nuestra evaluacion vemos que el Dataset luego de unidos los datos de transacciones y de identidades, tienen muchos datos NaN, nuestro aproach inicial fue de reducir las columnas con alto nive de % NaN y luego limpiar las filas. Esto nos dejaba con un dataset 40% aproximadamente mas chico y nos preocupaba que nuestro modelo se viera afectado en la prediccion por esto.

> Decidimos tomar el camina de auditar las columnas con un metodo denominado `lift missing values` que nos permite determinar la viabilidad de eliminar una columna evaluando que tanto impacto tiene en nuestro target. Si es cercana a 1 la variable podria se elminada, si es mayor o menor el valor NaN o la presencia de la columna con valor no NaN podria darnos pista de un fraude y ser valiosa.  

In [ ]:
result, drop_candidates = missing_audit(df_full_transaction, 'isFraud')
print(result)
print("Columnas candidatas a eliminar:", drop_candidates)

- ## 2 Feature engineering y data cleaning

> En nuestro archivo Pipelines las funciones v2 actualmente hacer un feature engineering llamando a la clase feature_engineering con el metodo build_preprocessor, trabajamos en especial de evitar la multicolinealidad para modelos como la Regresion Logistica y Imputamos valores NaN con la mediana en columnas numericas entre otros.

- ## 3 Preparacion de Variables para entrenamiento de Modelos 

> Nuetra intencion es realizar un entrenamiento de modelos para
- Regresion Logistica
- SVM
- XGBoost

> Para esto necesitamos que nuestro dataset este preparado para cada uno de nuestros modelos.

In [ ]:
# Dividimos fijamos nuestro target y separamos features categoricas y numericas
y = df_full_transaction['isFraud']
X = df_full_transaction.drop(columns=['isFraud'])

cat_cols, num_cols = split_features(X)
print(f"Categorical columns: {len(cat_cols)}")
print(f"Numerical columns: {len(num_cols)}")

In [ ]:
pipelines = {
    'Logistic Regression': logistic_pipeline_v2(df_full_transaction),
    'SVM': svm_pipeline_v2(df_full_transaction),
    'XGBoost': xgboost_pipeline_v2(df_full_transaction)
}

- ## 4 Entrenamiento de Modelos

> Entrenamos nuestros modelos aplicando la estrategia de StratifiedKFold, para evaluar nuestras metricas deseadas con nuestros pipelines en un KFold de 5.

In [ ]:
X_sub, _, y_sub, _ = train_test_split(X, y, 
                                      train_size=0.2,  # usar 20% de datos
                                      stratify=y,
                                      random_state=42)
results = evaluate_multiple(pipelines, X_sub, y_sub, cv_splits=3)

In [ ]:
results = evaluate_multiple(pipelines, X, y, cv_splits=5)
print(results)